In [1]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
bqm=BinaryQuadraticModel('BINARY')
numvar=[1,2,3,4]
x=["x1","x2","x3","x4"]
for i in numvar:
    bqm.add_variable(x[i-1],1)
    
    
# import local solver: simulated annealing
import neal
solver = neal.sampler.SimulatedAnnealingSampler()
sampleset = solver.sample_ising({'a': -0.5, 'b': 1.0}, {('a', 'b'): -1}, num_reads=10)
sampleset

SampleSet(rec.array([([-1, -1], -1.5, 1), ([-1, -1], -1.5, 1), ([-1, -1], -1.5, 1),
           ([-1, -1], -1.5, 1), ([-1, -1], -1.5, 1), ([-1, -1], -1.5, 1),
           ([-1, -1], -1.5, 1), ([-1, -1], -1.5, 1), ([-1, -1], -1.5, 1),
           ([-1, -1], -1.5, 1)],
          dtype=[('sample', 'i1', (2,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['a', 'b']), {'beta_range': [0.34657359027997264, 9.210340371976184], 'beta_schedule_type': 'geometric'}, 'SPIN')

In [422]:
# Program to include in the UI
import dimod
import neal
import numpy
sampler = neal.sampler.SimulatedAnnealingSampler()
bqm = dimod.BinaryQuadraticModel({'a': .6, 'b': -.9, 'c':.1, 'd':-.5, 'e':.9, 'f':0}, {('a', 'b'): 1,('b', 'c'): -1,('c', 'd'): +1,('d', 'e'): 1}, 0.0, dimod.BINARY)
# Run with default parameters
#sampleset = sampler.sample(bqm)
# Run with specified parameters
sampleset = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2],
                               num_reads=20, num_sweeps=1000,
                              beta_schedule_type='geometric', sorted_by='energy', initial_states_generator='random')
#sampleset = sampler.sample(bqm, seed=1234, num_reads=10, num_sweeps=1000, beta_schedule_type='geometric',sorted_by='energy')


print(sampleset)



    a  b  c  d  e  f energy num_oc.
2   0  1  1  0  0  0   -1.8       1
3   0  1  1  0  0  0   -1.8       1
4   0  1  1  0  0  0   -1.8       1
5   0  1  1  0  0  1   -1.8       1
6   0  1  1  0  0  1   -1.8       1
8   0  1  1  0  0  1   -1.8       1
9   0  1  1  0  0  0   -1.8       1
10  0  1  1  0  0  1   -1.8       1
12  0  1  1  0  0  0   -1.8       1
13  0  1  1  0  0  0   -1.8       1
14  0  1  1  0  0  0   -1.8       1
15  0  1  1  0  0  1   -1.8       1
16  0  1  1  0  0  1   -1.8       1
17  0  1  1  0  0  0   -1.8       1
19  0  1  1  0  0  1   -1.8       1
0   0  1  0  1  0  0   -1.4       1
18  0  1  0  1  0  0   -1.4       1
7   0  1  1  1  0  0   -1.3       1
11  0  1  1  1  0  1   -1.3       1
1   0  1  0  0  0  1   -0.9       1
['BINARY', 20 rows, 20 samples, 6 variables]


In [423]:
from collections import Counter

x=sampleset.record.sample
y=sampleset.record.energy
miny=min(y)
sortedy=sorted(y)
z=list(range(y.size))#creates a list of indices.
energyindexpair=[k for _, k in sorted(list(zip(y, z)))]# sorts (energy, index) pair based on energies and outputs ordered indices
sortedsample=[x[k] for k in energyindexpair] # creates a list of bitstrings sorted in increasing order of energy.

#find the number of lowest energy bitstrings
lowestenlistsize=0
for i in range(y.size):
    if sortedy[i]==miny:
        lowestenlistsize=i

# store the solutions (only the lowest energy bitstrings)
lowestenergysortedsample=sortedsample[0:lowestenlistsize+1]

#extract unique solutions and their frequency or probability
strsample=map(str,lowestenergysortedsample)
strsamplelist=list(map(str,lowestenergysortedsample))
struniqsols=list(set(strsample))
count=Counter(strsamplelist)
totalcount=sum(count.values())
solutionlist=list()
for i in range(len(struniqsols)):
    solutionlist.append(numpy.frombuffer(struniqsols[i].replace('[','').replace(']','').replace(" ",'').encode(), numpy.int8) - 48)
len(solutionlist)
probvector=[frequency/totalcount for frequency in count.values()]
print(probvector)

#Find marginal probability of solutions matching partial future projection vector.
futuresoln=[0,0,0,0,0,1]  #partial projection data
futuresolnpositions=[index for (index, item) in enumerate(futuresoln) if item == 1]
marginalprob=0
for i in range(len(solutionlist)):
    match=1
    for j in range(len(futuresolnpositions)):
        if solutionlist[i][futuresolnpositions[j]]==futuresoln[futuresolnpositions[j]]:
            match=match*1
        else:
            match=match*0
    if match==1:
        marginalprob=marginalprob+probvector[i]

        
#Check convergence

            

print(futuresoln)
print(solutionlist)
print(marginalprob)





[0.5333333333333333, 0.4666666666666667]
[0, 0, 0, 0, 0, 1]
[array([0, 1, 1, 0, 0, 1], dtype=int8), array([0, 1, 1, 0, 0, 0], dtype=int8)]
0.5333333333333333


In [309]:
strsamplelist[1].replace('[','').replace(']','').replace(" ",'')
numpy.frombuffer(strsamplelist[1].replace('[','').replace(']','').replace(" ",'').encode(), numpy.int8) - 48

array([0, 1, 1, 0, 0, 0], dtype=int8)

In [331]:
solutionlist

[array([0, 1, 1, 0, 0, 1], dtype=int8), array([0, 1, 1, 0, 0, 0], dtype=int8)]

In [435]:
bqm1=dimod.BinaryQuadraticModel('BINARY')
#bqm1.add_variable(6)
bqm1.add_quadratic_from_dense([[0,1,0,0,0,0],[0, 0, -1,0,0,0], [0, 0, 0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,0],[0,0,0,0,0,0]])
bqm1.add_linear_from_array([.6,-.9,.1,-.5,.9,0])
bqm1.set_linear_from_array([.2,-.9,.1,-.5,.9,0])
sampleset1=sampler.sample(bqm1, seed=1234, num_reads=20, num_sweeps=1000, beta_schedule_type='geometric',sorted_by='energy')


print(bqm1)

AttributeError: 'BinaryQuadraticModel' object has no attribute 'set_linear_from_array'

In [608]:
import dimod
import neal
import numpy
from collections import Counter
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

sampler = neal.sampler.SimulatedAnnealingSampler()

Cov=[[0,200,0,0,0,0],[0, 0, -40,0,0,0], [0, 0, 0,10,0,0],[0,0,0,0,20,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]# symmetric matrix
currentprice=[.6,-.9,.1,-.5,.9,0] #all elements of price vector should be >0 (not including 0)
totalcost=3                       # cost budget
numvar=len(currentprice)          # Number of assets
futuresoln=[0,0,0,0,1,0]          #partial projection data: O -> no info available, 1 -> must pick the asset
futuresolnpositions=[index for (index, item) in enumerate(futuresoln) if item == 1]
convergence=0
Pconvergence=0.9

#Calculate input quadratic and linear biases for the BQM
lagmultiplier=1
rcost=totalcost-sum([currentprice[k]*futuresoln[k] for k in range(numvar)]) #residual cost
constraintmatrix=[[0 for j in range(numvar)] for i in range(numvar)] #symmetrix matrix
for i in range(numvar):
    for j in range(numvar):
        if i==j:
            constraintmatrix[i][i]=lagmultiplier*(1-futuresoln[i])*(currentprice[i]*currentprice[i]-2*currentprice[i]*rcost)
        else:
            constraintmatrix[i][j]=lagmultiplier*(1-futuresoln[i])*(1-futuresoln[j])*2*currentprice[i]*currentprice[j]

#totalbiasmatrix and offset
totalbiasmatrix=[[Cov[i][j]+constraintmatrix[i][j] for j in range(numvar)] for i in range(numvar)]
quadraticbiasmatrix=[[0 for j in range(numvar)] for i in range(numvar)]
for i in range(numvar):
    for j in range(numvar):
        if j>i:
            quadraticbiasmatrix[i][j]=totalbiasmatrix[i][j]

linearbiasvector=[0 for i in range(numvar)]
for i in range(numvar):
    linearbiasvector[i]=totalbiasmatrix[i][i]

costoffset=rcost*rcost

# create BQM Object
bqm1=dimod.BinaryQuadraticModel('BINARY')
bqm1.add_quadratic_from_dense(quadraticbiasmatrix)
bqm1.add_linear_from_array(linearbiasvector)
bqm1.offset+=costoffset




while convergence!=1:
    
    # Sample from BQM
    sampleset1=sampler.sample(bqm1, seed=1234, num_reads=100, num_sweeps=1000, beta_schedule_type='geometric',sorted_by='energy')



    x=sampleset1.record.sample
    y=sampleset1.record.energy
    miny=min(y)
    sortedy=sorted(y)
    z=list(range(y.size))#creates a list of indices.
    energyindexpair=[k for _, k in sorted(list(zip(y, z)))]# sorts (energy, index) pair based on energies and outputs ordered indices
    sortedsample=[x[k] for k in energyindexpair] # creates a list of bitstrings sorted in increasing order of energy.

    # find the number of lowest energy bitstrings
    lowestenlistsize=0
    for i in range(y.size):
        if sortedy[i]==miny:
            lowestenlistsize=i

    # store the solutions (only the lowest energy bitstrings)
    lowestenergysortedsample=sortedsample[0:lowestenlistsize+1]

    # extract unique solutions and their frequency or probability
    strsample=map(str,lowestenergysortedsample)
    strsamplelist=list(map(str,lowestenergysortedsample))
    struniqsols=list(set(strsample))
    count=Counter(strsamplelist)
    totalcount=sum(count.values())
    solutionlist=list()
    for i in range(len(struniqsols)):
        solutionlist.append(numpy.frombuffer(struniqsols[i].replace('[','').replace(']','').replace(" ",'').encode(), numpy.int8) - 48)
    probvector=[frequency/totalcount for frequency in count.values()]
    print(probvector)
    print(solutionlist)
    print("----")
    
    #Find marginal probability of solutions matching partial future projection vector.
    marginalprob=0
    numsolutions=len(solutionlist)
    for i in range(len(solutionlist)):
        match=1
        for j in range(len(futuresolnpositions)):
            if solutionlist[i][futuresolnpositions[j]]==futuresoln[futuresolnpositions[j]]:
                match=match*1
            else:
                match=match*0
        if match==1:
            marginalprob=marginalprob+probvector[i]
            
    # Check for convergence
    if marginalprob>=Pconvergence:
        convergence=1
    else:
        localfieldincrement=[0 for k in range(numvar)]
        fieldweightingfactor=[currentprice[k]/totalcost for k in range(numvar)]
        averagesol=[0 for i in range(numvar)]
        sumsol=[0 for i in range(numvar)]
        delh=-0.1
        for i in range(numsolutions):
            sumsol=[sum(k) for k in zip(sumsol,solutionlist[i])]
        averagesol=[(1/numsolutions)*k for k in sumsol]
        
        localfieldincrement=[futuresoln[k]*(1-averagesol[k])*(fieldweightingfactor[k]+.5)*delh for k in range(numvar)]
        print(localfieldincrement)
    #update local fields in the BQM for the next iteration
    bqm1.add_linear_from_array(localfieldincrement)

print(solutionlist)
print(totalbiasmatrix)
print(quadraticbiasmatrix)
print(linearbiasvector)
print(bqm1.offset)

[0.32, 0.25, 0.23, 0.2]
[array([0, 1, 1, 0, 0, 1], dtype=int8), array([0, 1, 1, 0, 1, 1], dtype=int8), array([0, 1, 1, 0, 1, 0], dtype=int8), array([0, 1, 1, 0, 0, 0], dtype=int8)]
----
[-0.0, -0.0, -0.0, -0.0, -0.04000000000000001, -0.0]
[0.52, 0.48]
[array([0, 1, 1, 0, 1, 1], dtype=int8), array([0, 1, 1, 0, 1, 0], dtype=int8)]
----
[array([0, 1, 1, 0, 1, 1], dtype=int8), array([0, 1, 1, 0, 1, 0], dtype=int8)]
[[-2.16, 198.92, 0.12, -0.6, 0.0, 0.0], [-1.08, 4.59, -40.18, 0.9, 0.0, 0.0], [0.12, -0.18000000000000002, -0.41000000000000003, 9.9, 0.0, 0.0], [-0.6, 0.9, -0.1, 2.35, 20.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[0, 198.92, 0.12, -0.6, 0.0, 0.0], [0, 0, -40.18, 0.9, 0.0, 0.0], [0, 0, 0, 9.9, 0.0, 0.0], [0, 0, 0, 0, 20.0, 0.0], [0, 0, 0, 0, 0, 0.0], [0, 0, 0, 0, 0, 0]]
[-2.16, 4.59, -0.41000000000000003, 2.35, 0.0, 0.0]
4.41


In [520]:
bqm1.offset += -2

In [521]:
bqm1

BinaryQuadraticModel({0: 0.6, 1: -0.9, 2: 0.1, 3: -0.5, 4: 0.9, 5: -0.05}, {(1, 0): 1.0, (2, 1): -1.0, (3, 2): 1.0, (4, 3): 1.0}, 0.0, 'BINARY')

In [582]:
constraintmatrix=[[0 for j in range(numvar)] for i in range(numvar)]
for i in range(numvar):
    for j in range(numvar):
        if i==j:
            constraintmatrix[i][i]=(1-futuresoln[i])*(currentprice[i]*currentprice[i]-2*currentprice[i]*rcost)
        else:
            constraintmatrix[i][j]=(1-futuresoln[i])*(1-futuresoln[j])*2*currentprice[i]*currentprice[j]

print(constraintmatrix)

[[-3.2399999999999998, -1.08, 0.12, -0.6, 1.08, 0.0], [-1.08, 6.210000000000001, -0.18000000000000002, 0.9, -1.62, -0.0], [0.12, -0.18000000000000002, -0.5900000000000001, -0.1, 0.18000000000000002, 0.0], [-0.6, 0.9, -0.1, 3.25, -0.9, -0.0], [1.08, -1.62, 0.18000000000000002, -0.9, -4.59, 0.0], [0.0, -0.0, 0.0, -0.0, 0.0, 0.0]]
